In [1]:
import os
import xarray as xr
import rasterio as rio
import numpy as np
import dask
import dask.array as da
from dask.distributed import Client
from dask import delayed
from pyproj import Proj, Transformer, CRS
from tqdm.auto import tqdm
from pathlib import Path
from wrf import PolarStereographic

# local
from config import models, scenarios, variable_di, precision_di, months, unit_di, summary_di, mo_names


In [2]:

# Move the below block to config and let ist source the env var
DATA_DIR = Path(os.getenv("DATA_DIR") or "/atlas_scratch/cparr4/ncar_replacement_data")
DATA_DIR.mkdir(exist_ok=True, parents=True)

OUTPUT_DIR = Path(os.getenv("OUTPUT_DIR") or "/atlas_scratch/cparr4/AK_NCAR_12km_decadal_means_of_monthly_summaries")
OUTPUT_DIR.mkdir(exist_ok=True, parents=True)



In [3]:
paths = list(DATA_DIR.glob("*ws*.nc"))
assert len(paths) == 3000 # value learned from EDA notebook

In [4]:
process_group_di = {}
for model in models:
    process_group_di[model] = {}
    for scenario in scenarios:
        process_group_di[model][scenario] = []

for fp in paths:
    fp_model = fp.name.split("_")[0]
    fp_scenario = fp.name.split("_")[1]
    process_group_di[fp_model][fp_scenario].append(fp)

for model in models:
    for scenario in scenarios:
        assert len(process_group_di[model][scenario]) == 150

In [5]:
def mfload_all_netcdf_data(paths):
    with xr.open_mfdataset(paths, combine="nested", concat_dim=["time"]) as datacube:
        return datacube

In [6]:
def project_datacube(datacube):
    
    wrf_proj_str = PolarStereographic(**{"TRUELAT1": 64, "STAND_LON": -150}).proj4()
    wrf_proj = Proj(wrf_proj_str)
    wgs_proj = Proj(proj='latlong', datum='WGS84')
    
    transformer = Transformer.from_proj(wgs_proj, wrf_proj)
    e, n = transformer.transform(-150, 64)
    # Grid parameters
    dx, dy = 12000, 12000
    ny, nx = datacube.longitude.shape[1:]
    
    # Down left corner of the domain
    x0 = -(nx-1) / 2. * dx + e
    y0 = -(ny-1) / 2. * dy + n
    # 2d grid
    x = np.arange(nx) * dx + x0
    y = np.arange(ny) * dy + y0
    
    projected_datacube = datacube.assign_coords({"y": ("y", y), "x": ("x", x)})
    
    # Output geotiff creation profile params  
    width = datacube.x.shape[0]
    height = datacube.y.shape[0]
    
    # west and north
    west = x0 - dx / 2
    north = y[-1] + dy / 2
    out_transform = rio.transform.from_origin(west, north, dx, dy)
    
    wrf_raster_profile = {
        "driver": "GTiff",
        "crs": CRS.from_proj4(wrf_proj_str),
        "transform": out_transform,
        "width": width,
        "height": height,
        "count": 1,
        "dtype": np.float32,
        "nodata": -9999,
        "tiled": False,
        "compress": "lzw",
        "interleave": "band",
    }
    
    datacube.close()
    
    return projected_datacube, wrf_raster_profile

In [7]:
def list_years_in_decade(start_year):
    """
    Given a starting year, return a list of ten consecutive years representing a decade.

    Args:
        start_year (int): Starting year of the decade.

    Returns:
        list: A list of ten consecutive years.
    """
    return list(range(start_year, start_year + 10))


def slice_by_decade(datacube, decade_start_year):
    """
    Given a xarray datacube and a starting year, slice the datacube to select only
    the data from the decade corresponding to the starting year.

    Args:
        datacube (xarray.Dataset): A xarray datacube containing time series data.
        decade_start_year (int): Starting year of the decade to be sliced.

    Returns:
        xarray.Dataset: A sliced xarray datacube containing only the data from the
        specified decade.
    """
    years = list_years_in_decade(decade_start_year)
    decade_slice = datacube.isel(time=datacube.time.dt.year.isin(years))
    return decade_slice

In [8]:
def compute_monthly_summaries(decade_slice, climvar):
    
    summary_func = variable_di["ws"][climvar]
    out = (
        decade_slice[climvar]
        .resample(time="1M")
        .reduce(summary_func)
        .groupby("time.month")
        .reduce(np.mean)  # decadal summary is always a mean
    )
    dec_mean_monthly_summary = out.compute()
    return dec_mean_monthly_summary


In [9]:
def array_from_monthly_summary(dec_mean_monthly_summary, climvar, month):
    
    # we lose the orientation from xr and it flips upside down
    data = np.flipud(dec_mean_monthly_summary.sel(month=month).data)
    # round to sensible precision levels
    data = data.round(precision_di[climvar])
    # set nodata values to -9999
    data = np.nan_to_num(data, nan=-9999.0)
    return data

In [10]:
def make_output_filename(climvar, model, scenario, month, start_year):
    
    units = unit_di[climvar]
    mo_summary_func = summary_di[climvar]
    out_filename = f"{climvar.lower()}_{units}_{model}_{scenario}_{mo_names[month]}_{mo_summary_func}_{start_year}-{start_year + 10}_mean.tif"
    return out_filename


In [11]:
def write_raster_to_disk(out_filename, raster_profile, raster_data):
    with rio.open(
        OUTPUT_DIR / out_filename, "w", **raster_profile
    ) as dst:
        dst.write(raster_data, 1)

In [12]:
# create dask client
client = Client()
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 8
Total threads: 32,Total memory: 251.72 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:33263,Workers: 8
Dashboard: http://127.0.0.1:8787/status,Total threads: 32
Started: Just now,Total memory: 251.72 GiB
Comm: tcp://127.0.0.1:42445,Total threads: 4
Dashboard: http://127.0.0.1:40084/status,Memory: 31.46 GiB
Nanny: tcp://127.0.0.1:41243,


In [12]:
%%time

for model in models:
    for scenario in scenarios:
        ncube = mfload_all_netcdf_data(process_group_di[model][scenario])
        projcube, wrf_raster_profile = project_datacube(ncube)

        # change to source start and stop from config, so users could run "just historical" or something like that
        # also change increment for 20 year summaries, climos, etc.

        for decade_start in tqdm(range(1950, 2100, 10), desc=f"Processing {model} {scenario} data..."):
            decade_slice = slice_by_decade(projcube, decade_start)
            
            for climvar in tqdm(variable_di["ws"].keys(), desc=f"Processing {decade_start}s"):
                decadal_means_of_monthly_summaries = compute_monthly_summaries(decade_slice, climvar)
                
                for month in months:
                    month_array = array_from_monthly_summary(decadal_means_of_monthly_summaries, climvar, month)
                    output_filename = make_output_filename(climvar, model, scenario, month, decade_start)
                    write_raster_to_disk(output_filename, wrf_raster_profile, month_array)

            projcube.close()

Processing ACCESS1-3 rcp45 data...:   0%|          | 0/15 [00:00<?, ?it/s]

Processing 1950s:   0%|          | 0/5 [00:00<?, ?it/s]

Processing 1960s:   0%|          | 0/5 [00:00<?, ?it/s]

Processing 1970s:   0%|          | 0/5 [00:00<?, ?it/s]

Processing 1980s:   0%|          | 0/5 [00:00<?, ?it/s]

...

Processing 2060s:   0%|          | 0/5 [00:00<?, ?it/s]

Processing 2070s:   0%|          | 0/5 [00:00<?, ?it/s]

Processing 2080s:   0%|          | 0/5 [00:00<?, ?it/s]

Processing 2090s:   0%|          | 0/5 [00:00<?, ?it/s]

CPU times: user 2h 33min 48s, sys: 21min 45s, total: 2h 55min 34s
Wall time: 2h 3min 9s


In [ ]:
# see tbec repo for setting env vars for the input and output dirs
# pipeline seems to be working, add loop to roll through model and scenarios
# add notebook to reproject to 3338
# contact NCAR about the missing data for the met files, and the extreme precip blob